In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf data
!unzip drive/MyDrive/data.zip -d data
!mv data/content/* data
!rm -rf data/content

Archive:  drive/MyDrive/data.zip
  inflating: data/content/0_456_masked.png  
  inflating: data/content/0_457_masked.png  
  inflating: data/content/0_458_masked.png  
  inflating: data/content/0_459_masked.png  
  inflating: data/content/0_460_masked.png  
  inflating: data/content/0_461_masked.png  
  inflating: data/content/0_462_masked.png  
  inflating: data/content/0_463_masked.png  
  inflating: data/content/0_464_masked.png  
  inflating: data/content/0_465_masked.png  
  inflating: data/content/0_466_masked.png  
  inflating: data/content/0_467_masked.png  
  inflating: data/content/0_468_masked.png  
  inflating: data/content/0_469_masked.png  
  inflating: data/content/0_470_masked.png  
  inflating: data/content/0_471_masked.png  
  inflating: data/content/0_472_masked.png  
  inflating: data/content/0_473_masked.png  
  inflating: data/content/0_474_masked.png  
  inflating: data/content/0_475_masked.png  
  inflating: data/content/0_476_masked.png  
  inflating: data/cont

In [ ]:
import os
import shutil

for i in range(0, 6):
  if not os.path.exists(f'data/{i}'):
    os.mkdir(f'data/{i}')
  for item in os.listdir('data'):
    if item.startswith(str(-2)): print(item, os.path.isfile(f'data/{item}'), )
    if os.path.isfile(f'data/{item}') and item.startswith(str(i)):
      shutil.move(f'data/{item}', f'data/{i}/{item}')

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
def get_mean_std(data_dir):
    '''
    이미지 정규화 시 성능 향상 , 평균과 표준편차로 정규화 실행
    data_dir = 이미지 들어있는 폴더 path
    '''
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    dataset = datasets.ImageFolder(os.path.join(f'./{data_dir}'), transform)
    print("데이터 정보", dataset)

    meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in dataset]
    stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in dataset]

    meanR = np.mean([m[0] for m in meanRGB])
    meanG = np.mean([m[1] for m in meanRGB])
    meanB = np.mean([m[2] for m in meanRGB])

    stdR = np.mean([s[0] for s in stdRGB])
    stdG = np.mean([s[1] for s in stdRGB])
    stdB = np.mean([s[2] for s in stdRGB])
    print("평균",meanR, meanG, meanB)
    print("표준편차",stdR, stdG, stdB)

# train data, test data 다르게 nomalization 적용하려면 data_dir 바꾸세요.
data_dir = "data"
get_mean_std(data_dir)


데이터 정보 Dataset ImageFolder
    Number of datapoints: 1216
    Root location: ./data
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
평균 0.14603005 0.14603005 0.14603005
표준편차 0.1641779 0.1641779 0.1641779


In [ ]:
from torch.utils.data import random_split

dataset = ImageFolder(root='data', transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.14603005, 0.14603005 , 0.14603005], [0.1641779, 0.1641779, 0.1641779 ])]))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
from torchvision import models
from torch import nn

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, 1)
model = model.to(device)

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
import torch
import torch.nn as nn
from torch import optim

lr = 1e-6
num_epochs = 30
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_function = nn.MSELoss().to(device)

params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    'test_dataloader': val_dataloader,
    'device':device
}

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=5)

In [ ]:
def train(model, params):
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    test_dataloader=params["test_dataloader"]
    device=params["device"]

    best_loss=float('inf')
    early_stop_cnt = 0

    for epoch in range(0, num_epochs):
      for i, data in enumerate(train_dataloader, 0):
        # train dataloader 로 불러온 데이터에서 이미지와 라벨을 분리
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.float()
        labels = labels.to(device)

        # 이전 batch에서 계산된 가중치를 초기화
        optimizer.zero_grad()

        # forward + back propagation 연산
        outputs = model(inputs)
        train_loss = loss_function(outputs, labels)
        train_loss.backward()
        optimizer.step()

      # test accuracy 계산
      total = 0
      correct = 0
      accuracy = []
      for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.float()
        labels = labels.to(device)

        # 결과값 연산
        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        test_loss = loss_function(outputs, labels).item()
        accuracy.append(100 * correct/total)

      scheduler.step(test_loss)
      if test_loss < best_loss:
        best_loss = test_loss
        early_stop_cnt = 0
        torch.save(model.state_dict(), 'best_model.pt')

      else: early_stop_cnt += 1

      if early_stop_cnt >= 10:
        break

      if scheduler.num_bad_epochs > scheduler.patience:
        print(f'Early stopping at epoch {epoch}...')
        break

      # 학습 결과 출력
      print('Epoch: %d/%d, Train loss: %.6f, Test loss: %.6f, Accuracy: %.2f, Learning Rate: %.9f' %(epoch+1, num_epochs, train_loss.item(), test_loss, 100*correct/total, optimizer.param_groups[0]['lr']))

In [ ]:
train(model, params)

Epoch: 1/30, Train loss: 11.986035, Test loss: 11.912588, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 2/30, Train loss: 11.089191, Test loss: 6.516188, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 3/30, Train loss: 7.503634, Test loss: 2.448357, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 4/30, Train loss: 12.304681, Test loss: 4.132693, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 5/30, Train loss: 4.174641, Test loss: 2.285907, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 6/30, Train loss: 5.874025, Test loss: 1.994257, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 7/30, Train loss: 7.578540, Test loss: 7.431650, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 8/30, Train loss: 5.821738, Test loss: 5.210853, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 9/30, Train loss: 5.868315, Test loss: 6.457963, Accuracy: 10.66, Learning Rate: 0.000001000
Epoch: 10/30, Train loss: 4.668549, Test loss: 4.838428, Accuracy: 10.66, Learning Rate: 0.00000

In [ ]:
!unzip drive/MyDrive/Archive.zip -d test

Archive:  drive/MyDrive/Archive.zip
   creating: test/-1/
  inflating: test/__MACOSX/._-1      
  inflating: test/-1/-1_2_masked.JPG  
  inflating: test/__MACOSX/-1/._-1_2_masked.JPG  
  inflating: test/-1/-1_1_masked.JPG  
  inflating: test/__MACOSX/-1/._-1_1_masked.JPG  
  inflating: test/-1/-1_3_masked.JPG  
  inflating: test/__MACOSX/-1/._-1_3_masked.JPG  
   creating: test/-2/
  inflating: test/__MACOSX/._-2      
  inflating: test/-2/-2_3_masked.JPG  
  inflating: test/__MACOSX/-2/._-2_3_masked.JPG  
  inflating: test/-2/-2_2_masked.JPG  
  inflating: test/__MACOSX/-2/._-2_2_masked.JPG  
   creating: test/0/
  inflating: test/__MACOSX/._0       
  inflating: test/0/0_1_masked.JPG   
  inflating: test/__MACOSX/0/._0_1_masked.JPG  
  inflating: test/0/0_3_masked.JPG   
  inflating: test/__MACOSX/0/._0_3_masked.JPG  
  inflating: test/0/0_2_masked.JPG   
  inflating: test/__MACOSX/0/._0_2_masked.JPG  
   creating: test/1/
  inflating: test/__MACOSX/._1       
  inflating: test/1/1_1

In [ ]:
!rm -rf test/_MACOSX

In [ ]:
import torch
import numpy as np

# 데이터 불러오기
# ... (데이터 로딩 코드 작성)

# 모델 불러오기
model = torch.load('path/to/saved_model.pt')

# 모델 평가 모드로 설정
model.eval()

# 예측 계산
with torch.no_grad():
    y_pred = model(X)

# R2 평가 계산
y_true = torch.tensor(y)  # 실제값을 텐서로 변환
r2_score = 1 - torch.nn.functional.mse_loss(y_pred, y_true)

# R2 점수 출력
print("R2 점수:", r2_score.item())


In [ ]:
!cp /content/best_model.pt /content/drive/MyDrive/best_model.pt